In [176]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

通过查找源数据集的标签来为测试集加上标签，以便后续测试。

# TA版数据集

In [177]:
trainSet_origin = pd.read_csv('data\\train.csv')
testSet_origin = pd.read_csv('data\\test.csv')

trainSet_origin.shape, testSet_origin.shape

((16659, 9), (742, 9))

In [178]:
trainSet_origin.head()
testSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,1,2011/1/1,0,1,9.84,14.395,81,0,16
1,2,2011/1/1,1,1,9.02,13.635,80,0,40
2,3,2011/1/1,2,1,9.02,13.635,80,0,32
3,4,2011/1/1,3,1,9.84,14.395,75,0,13
4,5,2011/1/1,4,1,9.84,14.395,75,0,1


,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,?


# 数据源数据集

In [179]:
allData = pd.read_csv('data\\Bike-Sharing-Dataset\\hour.csv')
allData.shape

allData.head()

(17379, 17)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [180]:
allData.drop(["season", "yr", "mnth", "holiday", "weekday", "workingday", "casual", "registered"], axis=1, inplace=True)

allData.head()
testSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,1,2011-01-01,0,1,0.24,0.2879,0.81,0.0,16
1,2,2011-01-01,1,1,0.22,0.2727,0.80,0.0,40
2,3,2011-01-01,2,1,0.22,0.2727,0.80,0.0,32
3,4,2011-01-01,3,1,0.24,0.2879,0.75,0.0,13
4,5,2011-01-01,4,1,0.24,0.2879,0.75,0.0,1


,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,?


发现原始数据集的时间特征表示形式不一致。

In [181]:
allData['dteday'] = allData['dteday'].apply(lambda x: x.replace("-","/"))

In [182]:
for i in range(1, 10):
    allData['dteday'] = allData['dteday'].apply(lambda x: x.replace("/0"+str(i), "/"+str(i)))

In [183]:
allData.head()
testSet_origin.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,1,2011/1/1,0,1,0.24,0.2879,0.81,0.0,16
1,2,2011/1/1,1,1,0.22,0.2727,0.80,0.0,40
2,3,2011/1/1,2,1,0.22,0.2727,0.80,0.0,32
3,4,2011/1/1,3,1,0.24,0.2879,0.75,0.0,13
4,5,2011/1/1,4,1,0.24,0.2879,0.75,0.0,1


,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,?


# 得到测试集在原始数据集的位置

In [184]:
start = testSet_origin.loc[0, :]['instant']
end = testSet_origin.loc[testSet_origin.shape[0]-1, :]['instant']

start, end

(16638, 17379)

In [185]:
testSet_origin.shape

(742, 9)

In [186]:
testSet_answer = allData.loc[start-1:end+1, :].reset_index()
testSet_answer.drop(["index"], axis=1, inplace=True)
testSet_answer.shape

(742, 9)

In [187]:
testSet_origin.head()
testSet_answer.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,?


,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,0.26,0.3030,0.81,0.0000,108
1,16639,2012/12/1,1,1,0.26,0.3030,0.81,0.0000,69
2,16640,2012/12/1,2,2,0.26,0.3030,0.81,0.0000,50
3,16641,2012/12/1,3,2,0.26,0.2727,0.81,0.1343,15
4,16642,2012/12/1,4,1,0.26,0.2879,0.81,0.0896,5


In [188]:
testSet_answer['temp'] = testSet_answer['temp']*41
testSet_answer['atemp'] = testSet_answer['atemp']*50
testSet_answer['hum'] = testSet_answer['hum']*100
testSet_answer['windspeed'] = testSet_answer['windspeed']*67

In [189]:
testSet_origin.head()
testSet_answer.head()

,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81,0.0000,?
1,16639,2012/12/1,1,1,10.66,15.150,81,0.0000,?
2,16640,2012/12/1,2,2,10.66,15.150,81,0.0000,?
3,16641,2012/12/1,3,2,10.66,13.635,81,8.9981,?
4,16642,2012/12/1,4,1,10.66,14.395,81,6.0032,?


,instant,dteday,hr,weathersit,temp,atemp,hum,windspeed,cnt
0,16638,2012/12/1,0,1,10.66,15.150,81.0,0.0000,108
1,16639,2012/12/1,1,1,10.66,15.150,81.0,0.0000,69
2,16640,2012/12/1,2,2,10.66,15.150,81.0,0.0000,50
3,16641,2012/12/1,3,2,10.66,13.635,81.0,8.9981,15
4,16642,2012/12/1,4,1,10.66,14.395,81.0,6.0032,5


# 检查testSet_origin和testSet_answer是否在所有特征上的数据都一致

In [190]:
totSampleNum = testSet_origin.shape[0]
totSampleNum

742

首先检查离散变量

In [191]:
for feature in ['instant', 'dteday', 'hr', 'weathersit']: 
    check_statement = testSet_origin[feature] == testSet_answer[feature]
    assert(Counter(list(check_statement))[True] == totSampleNum)
    print("success match "+str(feature))

success match instant
success match dteday
success match hr
success match weathersit


接着检查连续变量

In [192]:
for feature in ['temp', 'atemp', 'hum', 'windspeed']: 
    assert(np.allclose(testSet_origin[feature], testSet_answer[feature]))
    print("success match "+str(feature))

success match temp
success match atemp
success match hum
success match windspeed


# 保存测试集标签

很好，现在保证了testSet_origin和testSet_answer是否在所有特征上的数据都一致，可以保存标签用于测试集了。

In [193]:
testSet_answer['cnt'].shape

(742,)

In [195]:
testSet_answer['cnt'].to_csv("data\\test_label.csv", index=False, header=False)